In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
import datetime as dt
import pytz
from retry_requests import retry
from metpy.calc import wind_components
from metpy.units import units
from datetime import timedelta

In [2]:

def get_ordered_hourly_variables(params):
    return params.get("hourly", [])

#setting up API client
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)




In [ ]:
def get_reading(url, latitude, longtitute, hourly, past_days, future_days):
  params = {
    "latitude": latitude,
    "longitude": longtitute,
    "hourly": hourly,
    "past_days": past_days,
    "forecast_days": future_days
  }

  response = openmeteo.weather_api(url, params=params)[0]
  hourly = response.Hourly()
  hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )
  }
  for i, var_name in enumerate(get_ordered_hourly_variables(params)):
    hourly_data[var_name] = hourly.Variables(i).ValuesAsNumpy()

  #Modify df
  hourly_df = pd.DataFrame(data=hourly_data)
  hourly_df['Date/Time'] = pd.to_datetime(hourly_df['date'], errors='coerce')
  hourly_df.set_index('Date/Time', inplace=True)
  hourly_df.drop(columns=['date'], inplace = True)

  return hourly_df

In [3]:
def get_reading(url, latitude, longtitute, hourly, past_days, future_days):
    params = {
        "latitude": latitude,
        "longitude": longtitute,
        "hourly": hourly,
        "past_days": past_days,
        "forecast_days": future_days
    }

    response = openmeteo.weather_api(url, params=params)[0]
    hourly = response.Hourly()

    # Convert from UTC to UTC+3
    start_time = pd.to_datetime(hourly.Time(), unit="s", utc=True) + pd.Timedelta(hours=3)
    end_time = pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True) + pd.Timedelta(hours=3)

    hourly_data = {
        "date": pd.date_range(
            start=start_time,
            end=end_time,
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )
    }

    for i, var_name in enumerate(get_ordered_hourly_variables(params)):
        hourly_data[var_name] = hourly.Variables(i).ValuesAsNumpy()

    hourly_df = pd.DataFrame(data=hourly_data)
    hourly_df['Date/Time'] = pd.to_datetime(hourly_df['date'], errors='coerce')
    hourly_df.set_index('Date/Time', inplace=True)
    hourly_df.drop(columns=['date'], inplace=True)

    return hourly_df

In [4]:
df = get_reading("https://api.open-meteo.com/v1/forecast",
            31.990534, 35.907787,
            ["temperature_2m", "dew_point_2m", "relative_humidity_2m",
           "pressure_msl", "wind_speed_10m", "wind_direction_10m"], 5,
            0)


In [5]:
df

,temperature_2m,dew_point_2m,relative_humidity_2m,pressure_msl,wind_speed_10m,wind_direction_10m
Date/Time,,,,,,
2025-05-04 03:00:00+00:00,12.440000,3.900462,56.0,1011.799988,3.671294,281.309906
2025-05-04 04:00:00+00:00,11.639999,4.369053,61.0,1011.599976,3.240000,270.000000
2025-05-04 05:00:00+00:00,11.340000,4.317975,62.0,1011.500000,3.600000,270.000000
2025-05-04 06:00:00+00:00,10.840000,4.297762,64.0,1011.799988,3.617955,264.289490
2025-05-04 07:00:00+00:00,11.889999,5.067717,63.0,1012.400024,4.693825,265.601379
...,...,...,...,...,...,...
2025-05-08 22:00:00+00:00,21.639999,2.339473,28.0,1013.099976,6.480000,360.000000
2025-05-08 23:00:00+00:00,21.090000,0.833397,26.0,1013.000000,5.588703,14.931467
2025-05-09 00:00:00+00:00,19.590000,1.063712,29.0,1014.500000,4.582052,44.999897


In [6]:
df.rename(columns={
    "dew_point_2m": "Air Dew Point",
    "temperature_2m": "Air Temperature (OC)",
    "relative_humidity_2m": "Humidity %",
    "pressure_msl": "Atmospheric Pressure",
    "wind_speed_10m": "Wind Speed (MPS)",
    "wind_direction_10m": "Wind Direction (Degrees)"
}, inplace=True)

df.index = df.index.tz_convert(None)

u, v = wind_components(
    df['Wind Speed (MPS)'].values * units('m/s'),
    df['Wind Direction (Degrees)'].values * units.degree
)

df['Wind_U'] = u.magnitude  
df['Wind_V'] = v.magnitude
df.Wind_U = df.Wind_U.round(2)
df.Wind_V = df.Wind_V.round(2)

df.drop(columns=["Wind Speed (MPS)", "Wind Direction (Degrees)"], inplace=True)

In [8]:
df.tail()

,Air Temperature (OC),Air Dew Point,Humidity %,Atmospheric Pressure,Wind_U,Wind_V
Date/Time,,,,,,
2025-05-08 22:00:00,21.639999,2.339473,28.0,1013.099976,-0.00,-6.48
2025-05-08 23:00:00,21.090000,0.833397,26.0,1013.000000,-1.44,-5.40
2025-05-09 00:00:00,19.590000,1.063712,29.0,1014.500000,-3.24,-3.24
2025-05-09 01:00:00,19.539999,0.533527,28.0,1014.400024,-2.16,-2.88
2025-05-09 02:00:00,19.639999,-0.403245,26.0,1014.200012,-1.44,-1.44
